In [102]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set(style="white")

## Perfecting our prediction model: amenities vs. price
Our previous attempts at increasing the R value (see Amenity_count_vs_Price_2) overfit our model and had a limited increase in R value, and therefore could not be kept. This notebook attempts to increase the R value by using a purely linear relationship between amenities and price, but trying to control for different variables. 

Steps: 
    1. import data
    2. create dummy columns based on different values
        - Country
        - property type
        - room type 
    3. create different OLS models to see if holding different variables constant will affect the outcomes
`Accommodates` will always be in our models because it indicates how large the property is, which could have a strong effect on price.

### Step 1: importing data

In [103]:
df = pd.read_pickle("../../Datasets/Data_viz_project/airbnb_listings_price_normalized")

In [79]:
df.columns

Index(['Listing ID', 'Name', 'Host ID', 'Host Name', 'Host Response Rate',
       'Host Is Superhost', 'Host total listings count', 'Country', 'latitude',
       'longitude', 'Property type', 'Room type', 'Accommodates', 'Bathrooms',
       'Bedrooms', 'Amenities', 'Price', 'Minimum nights', 'Maximum nights',
       'Availability 365', 'Calendar last scraped', 'Number of reviews',
       'Last Review Date', 'Review Scores Rating', 'Review Scores Accuracy',
       'Review Scores Cleanliness', 'Review Scores Checkin',
       'Review Scores Communication', 'Review Scores Location',
       'Review Scores Value', 'Reviews per month', 'Amenity_Count',
       'Country_Average', 'Normalized'],
      dtype='object')

### Step 2: create dummy tables and merge on the new "index" column

In [80]:
df = df.reset_index()

In [81]:
dummy = pd.get_dummies(df["Country"], drop_first=True)

In [82]:
dummy = dummy.reset_index()

In [83]:
dummy_room_type = pd.get_dummies(df["Room type"], drop_first=True).reset_index()

In [84]:
dummy_property_type = pd.get_dummies(df["Property type"], drop_first=True).reset_index()

In [85]:
df_amenities = df_amenities.reset_index()

In [86]:
df = (df.merge(dummy, on="index")
      .merge(dummy_room_type, on="index")
      .merge(dummy_property_type, on="index")
      .drop(columns=("index")))

In [87]:
df.columns

Index(['Listing ID', 'Name', 'Host ID', 'Host Name', 'Host Response Rate',
       'Host Is Superhost', 'Host total listings count', 'Country', 'latitude',
       'longitude', 'Property type', 'Room type', 'Accommodates', 'Bathrooms',
       'Bedrooms', 'Amenities', 'Price', 'Minimum nights', 'Maximum nights',
       'Availability 365', 'Calendar last scraped', 'Number of reviews',
       'Last Review Date', 'Review Scores Rating', 'Review Scores Accuracy',
       'Review Scores Cleanliness', 'Review Scores Checkin',
       'Review Scores Communication', 'Review Scores Location',
       'Review Scores Value', 'Reviews per month', 'Amenity_Count',
       'Country_Average', 'Normalized', 'Austria', 'Belgium', 'Canada',
       'Denmark', 'France', 'Germany', 'Greece', 'Hong Kong', 'Ireland',
       'Italy', 'Netherlands', 'Spain', 'Switzerland', 'United Kingdom',
       'United States', 'Private room', 'Shared room', 'Bed & Breakfast',
       'Boat', 'Boutique hotel', 'Bungalow', 'Cabin', 

### Regression 1: Looking at accommodates 

In [88]:
df["Const"] = 1

In [89]:
X = df[["Amenity_Count", "Accommodates", "Const"]]
y = df["Normalized"]

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                 1.040e+05
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:42   Log-Likelihood:            -3.6332e+05
No. Observations:              380224   AIC:                         7.266e+05
Df Residuals:                  380221   BIC:                         7.267e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Amenity_Count     0.0056      0.000     33.623      0.000       0.005       0.006
Accommodates      0.2246      0.001    436.703      0.000       0.224       0.226
Const             0.1640      0.003     54.187      0.000       0.158       0.170
==============================================================================
Omnibus:                   268259.638   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10136729.673
Skew:                           2.947   Prob(JB):                         0.00
Kurtosis:                      27.599   Cond. No.                         52.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression 2: Looking at different country dummy variables

In [90]:
X = df[["Amenity_Count", "Accommodates", "Const", "Austria", 
        "Belgium", "Canada", "Denmark", 
        "France", "Germany", "Greece", 
        "Hong Kong", "Ireland", "Italy",
       "Netherlands", "Spain", "Switzerland",
       "United Kingdom", "United States"]]
y = df["Normalized"]

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                 1.273e+04
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:44   Log-Likelihood:            -3.6060e+05
No. Observations:              380224   AIC:                         7.212e+05
Df Residuals:                  380206   BIC:                         7.214e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Amenity_Count      0.0073      0.000     41.702      0.000       0.007       0.008
Accommodates       0.2291      0.001    443.640      0.000       0.228       0.230
Const              0.1313      0.005     27.796      0.000       0.122       0.141
Austria           -0.0211      0.008     -2.573      0.010      -0.037      -0.005
Belgium            0.0596      0.009      6.587      0.000       0.042       0.077
Canada            -0.0006      0.005     -0.112      0.911      -0.011       0.010
Denmark            0.1074      0.007     16.254      0.000       0.094       0.120
France             0.0690      0.005     14.317      0.000       0.060       0.078
Germany            0.1572      0.006     24.908      0.000       0.145       0.170
Greece            -0.1053      0.011     -9.796      0.000      -0.126      -0.084
Hong Kong          0.1062      0.011      9.700      0.000       0.085       0.128
Ireland            0.0465      0.009      4.947      0.000       0.028       0.065
Italy             -0.1181      0.005    -21.624      0.000      -0.129      -0.107
Netherlands        0.0836      0.006     13.449      0.000       0.071       0.096
Spain             -0.1128      0.005    -22.734      0.000      -0.122      -0.103
Switzerland        0.1693      0.016     10.839      0.000       0.139       0.200
United Kingdom     0.0326      0.005      6.917      0.000       0.023       0.042
United States     -0.0624      0.004    -14.951      0.000      -0.071      -0.054
==============================================================================
Omnibus:                   271221.057   Durbin-Watson:                   1.705
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10570324.751
Skew:                           2.987   Prob(JB):                         0.00
Kurtosis:                      28.130   Cond. No.                         301.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Although the R value is better in this model, it is still not of statistical significance. 

### Regression 3: Controlling for  GDP per capita

GDP per capita data pulled from Wikipedia. We used the 2018 World Bank data because it is the closest in time to our estimated time frame from the data: https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28PPP%29_per_capita

In [91]:
GDP = {'Australia':51663,
 'Austria':55455,
 'Belgium':51408,
 'Canada':48130,
 'Denmark':55671,
 'France':45342,
 'Germany':53075,
 'Greece':29592,
 'Hong Kong':64597,
 'Ireland':83203,
 'Italy':41830,
 'Netherlands':56329,
 'Spain':39715,
 'Switzerland':68061,
 'United Kingdom':45973,
 'United States':62795}

In [92]:
GDP_list = [GDP[x] for x in df["Country"].to_list()]

In [93]:
df = df.assign(GDP=GDP_list)

In [94]:
X = df[["Amenity_Count", "Accommodates", "Const","GDP"]]
y = df["Normalized"]

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                 6.934e+04
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:47   Log-Likelihood:            -3.6329e+05
No. Observations:              380224   AIC:                         7.266e+05
Df Residuals:                  380220   BIC:                         7.266e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Amenity_Count     0.0054      0.000     31.624      0.000       0.005       0.006
Accommodates      0.2249      0.001    436.089      0.000       0.224       0.226
Const             0.1264      0.006     20.310      0.000       0.114       0.139
GDP            7.726e-07   1.12e-07      6.914      0.000    5.54e-07    9.92e-07
==============================================================================
Omnibus:                   268382.526   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10167364.151
Skew:                           2.948   Prob(JB):                         0.00
Kurtosis:                      27.637   Cond. No.                     3.24e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.24e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

 Here we see that GDP is not as strongly corrolated at dummy variables for each country. We will choose to keep dummy country variables instead of the GDP.

### Regression 4: Controlling for Tourism

Data was pulled from the UN Tourism department: https://www.e-unwto.org/doi/pdf/10.18111/9789284421152
We are looking at the international tourist arrivals by 1,000 in 2017 (data starts on page 17)

In [95]:
Tourism = {'Australia':8815,
 'Austria':29460,
 'Belgium':8358,
 'Canada':20883,
 'Denmark':11743,
 'France':86918,
 'Germany':37452,
 'Greece':27194,
 'Hong Kong':27885,
 'Ireland':10338,
 'Italy':58253,
 'Netherlands':17924,
 'Spain':81869,
 'Switzerland':11133,
 'United Kingdom':37651,
 'United States':76941}

In [96]:
Tourism_list = [Tourism[x] for x in df["Country"].to_list()]

In [97]:
df = df.assign(Tourism=Tourism_list)

In [98]:
X = df[["Amenity_Count", "Accommodates", "Const", "Tourism"]]
y = df["Normalized"]

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                 6.992e+04
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:48   Log-Likelihood:            -3.6273e+05
No. Observations:              380224   AIC:                         7.255e+05
Df Residuals:                  380220   BIC:                         7.255e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Amenity_Count     0.0058      0.000     34.705      0.000       0.005       0.006
Accommodates      0.2258      0.001    438.694      0.000       0.225       0.227
Const             0.2231      0.003     64.191      0.000       0.216       0.230
Tourism        -1.23e-06   3.57e-08    -34.423      0.000    -1.3e-06   -1.16e-06
==============================================================================
Omnibus:                   269462.354   Durbin-Watson:                   1.687
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10305845.580
Skew:                           2.963   Prob(JB):                         0.00
Kurtosis:                      27.807   Cond. No.                     2.07e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

This model slightly increases the R value, but is still not of statistical significance. 

### Regression 5: Controlling for property and room type
Note: `Accommodates` and `Property Type` could be correlated, which could affect our R value. If our R value is over 0.7, we will take measures to ensure control for this correlation. 

In [99]:
X = df[["Amenity_Count", "Accommodates", "Const","Private room", "Shared room",'Bed & Breakfast',
       'Boat', 'Boutique hotel', 'Bungalow', 'Cabin', 'Camper/RV',
       'Casa particular', 'Castle', 'Cave', 'Chalet', 'Condominium', 'Dorm',
       'Earth House', 'Entire Floor', 'Guest suite', 'Guesthouse',
       'Heritage hotel (India)', 'Hostel', 'House', 'Hut', 'Igloo', 'In-law',
       'Island', 'Lighthouse', 'Loft', 'Nature lodge', 'Other',
       'Parking Space', 'Ryokan (Japan)', 'Serviced apartment', 'Tent',
       'Timeshare', 'Tipi', 'Townhouse', 'Train', 'Treehouse', 'Vacation home',
       'Villa', 'Yurt' 
       ]]
y = df["Normalized"]

model = sm.OLS(y, X).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     5718.
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:52   Log-Likelihood:            -3.5143e+05
No. Observations:              380224   AIC:                         7.029e+05
Df Residuals:                  380180   BIC:                         7.034e+05
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Amenity_Count              0.0045      0.000     27.301      0.000       0.004       0.005
Accommodates               0.1876      0.001    328.260      0.000       0.186       0.189
Const                      0.4114      0.003    121.334      0.000       0.405       0.418
Private room              -0.3404      0.002   -137.396      0.000      -0.345      -0.336
Shared room               -0.5812      0.009    -67.632      0.000      -0.598      -0.564
Bed & Breakfast            0.1033      0.007     14.648      0.000       0.090       0.117
Boat                       0.3850      0.023     17.055      0.000       0.341       0.429
Boutique hotel             0.3709      0.037      9.985      0.000       0.298       0.444
Bungalow                  -0.1500      0.019     -7.932      0.000      -0.187      -0.113
Cabin                     -0.0818      0.022     -3.796      0.000      -0.124      -0.040
Camper/RV                 -0.3108      0.037     -8.354      0.000      -0.384      -0.238
Casa particular           -0.1985      0.193     -1.030      0.303      -0.577       0.179
Castle                     0.3557      0.092      3.869      0.000       0.175       0.536
Cave                       0.1127      0.230      0.489      0.625      -0.339       0.564
Chalet                     0.0754      0.035      2.156      0.031       0.007       0.144
Condominium                0.1121      0.007     16.398      0.000       0.099       0.126
Dorm                      -0.2653      0.021    -12.440      0.000      -0.307      -0.224
Earth House               -0.0499      0.103     -0.484      0.628      -0.252       0.152
Entire Floor              -0.0235      0.431     -0.054      0.957      -0.869       0.822
Guest suite               -0.0639      0.028     -2.305      0.021      -0.118      -0.010
Guesthouse                -0.0975      0.016     -5.977      0.000      -0.130      -0.066
Heritage hotel (India)     0.3249      0.610      0.533      0.594      -0.870       1.520
Hostel                    -0.2354      0.035     -6.688      0.000      -0.304      -0.166
House                      0.0471      0.003     17.361      0.000       0.042       0.052
Hut                       -0.1243      0.127     -0.978      0.328      -0.374       0.125
Igloo                     -0.1268      0.352     -0.360      0.719      -0.817       0.563
In-law                    -0.2059      0.051     -4.049      0.000      -0.306      -0.106
Island                     0.1181      0.273      0.433      0.665      -0.416       0.653
Lighthouse                 0.2548      0.273      0.934      0.350      -0.280       0.789
Loft                       0.1489      0.009     16.943      0.000       0.132       0.166
Nature lodge              -0.4901      0.203     -2.411      0.016      -0.888  

We found that it does increase the R value, but is still a medium strength model. There are also a few downsides of having so many variables. Although it increases the R, many of these could interact (Accommodates and Room Type, for example). Additionally, it is difficult to know which of these have the strongest influcence. 

### Regression 6: Controlling for review score rating

In [100]:
X = df[["Amenity_Count", "Accommodates", "Const", "Review Scores Rating"]]
y = df["Normalized"]

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                 7.047e+04
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:53   Log-Likelihood:            -3.6220e+05
No. Observations:              380224   AIC:                         7.244e+05
Df Residuals:                  380220   BIC:                         7.244e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Amenity_Count            0.0042      0.000     24.869      0.000       0.004       0.005
Accommodates             0.2261      0.001    440.049      0.000       0.225       0.227
Const                   -0.3629      0.012    -31.502      0.000      -0.385      -0.340
Review Scores Rating     0.0059      0.000     47.396      0.000       0.006       0.006
==============================================================================
Omnibus:                   269407.901   Durbin-Watson:                   1.679
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10325255.128
Skew:                           2.962   Prob(JB):                         0.00
Kurtosis:                      27.832   Cond. No.                     1.07e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Regression 7: The combination model
Here we will try to combine the most successful models to see if we can achive an R value of over 0.4

In [101]:
X = df[["Amenity_Count", "Accommodates", "Const", "Austria", 
        "Belgium", "Canada", "Denmark", 
        "France", "Germany", "Greece", 
        "Hong Kong", "Ireland", "Italy",
       "Netherlands", "Spain", "Switzerland",
       "United Kingdom", "United States",
        "Private room", "Shared room",'Bed & Breakfast',
       'Boat', 'Boutique hotel', 'Bungalow', 'Cabin', 'Camper/RV',
       'Casa particular', 'Castle', 'Cave', 'Chalet', 'Condominium', 'Dorm',
       'Earth House', 'Entire Floor', 'Guest suite', 'Guesthouse',
       'Heritage hotel (India)', 'Hostel', 'House', 'Hut', 'Igloo', 'In-law',
       'Island', 'Lighthouse', 'Loft', 'Nature lodge', 'Other',
       'Parking Space', 'Ryokan (Japan)', 'Serviced apartment', 'Tent',
       'Timeshare', 'Tipi', 'Townhouse', 'Train', 'Treehouse', 'Vacation home',
       'Villa', 'Yurt' 
       ]]

y = df["Normalized"]

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Normalized   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     4386.
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        12:07:59   Log-Likelihood:            -3.4885e+05
No. Observations:              380224   AIC:                         6.978e+05
Df Residuals:                  380165   BIC:                         6.985e+05
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Amenity_Count              0.0055      0.000     32.172      0.000       0.005       0.006
Accommodates               0.1908      0.001    330.127      0.000       0.190       0.192
Const                      0.3990      0.005     80.283      0.000       0.389       0.409
Austria                   -0.0270      0.008     -3.367      0.001      -0.043      -0.011
Belgium                    0.0349      0.009      3.975      0.000       0.018       0.052
Canada                    -0.0232      0.005     -4.391      0.000      -0.033      -0.013
Denmark                    0.0528      0.006      8.163      0.000       0.040       0.066
France                    -0.0128      0.005     -2.663      0.008      -0.022      -0.003
Germany                    0.1788      0.006     28.870      0.000       0.167       0.191
Greece                    -0.1532      0.010    -14.637      0.000      -0.174      -0.133
Hong Kong                  0.1958      0.011     18.289      0.000       0.175       0.217
Ireland                    0.0886      0.009      9.716      0.000       0.071       0.106
Italy                     -0.1304      0.005    -24.030      0.000      -0.141      -0.120
Netherlands                0.0071      0.006      1.163      0.245      -0.005       0.019
Spain                     -0.1051      0.005    -21.600      0.000      -0.115      -0.096
Switzerland                0.1446      0.015      9.536      0.000       0.115       0.174
United Kingdom             0.0614      0.005     13.328      0.000       0.052       0.070
United States             -0.0468      0.004    -11.526      0.000      -0.055      -0.039
Private room              -0.3475      0.003   -137.528      0.000      -0.352      -0.343
Shared room               -0.5800      0.009    -67.708      0.000      -0.597      -0.563
Bed & Breakfast            0.1564      0.007     21.795      0.000       0.142       0.170
Boat                       0.3734      0.023     16.528      0.000       0.329       0.418
Boutique hotel             0.3821      0.037     10.355      0.000       0.310       0.454
Bungalow                  -0.1406      0.019     -7.471      0.000      -0.177      -0.104
Cabin                     -0.0758      0.021     -3.532      0.000      -0.118      -0.034
Camper/RV                 -0.2859      0.037     -7.731      0.000      -0.358      -0.213
Casa particular           -0.1333      0.192     -0.696      0.487      -0.509       0.242
Castle                     0.3933      0.091      4.306      0.000       0.214       0.572
Cave                       0.1530      0.229      0.668      0.504      -0.296       0.602
Chalet                     0.1369      0.035      3.933      0.000       0.069       0.205
Condominium                0.1256      0.007     18.280      0.000       0.112  

## This model is still not statistically relevant enough to base predictions off of. We decide to abandon this idea and move on to the US only data, to see if there is a stronger correlation between amenity and price within a smaller region. Our US data also allows us the ability to control for cities, which could increase the R value